Привет, это базовый пример использования библиотеки [Signess](https://github.com/ElishaFlacon/signess/tree/main/example/datasets)

Для начала нужно установить саму библиотеку:

In [ ]:
!pip install --upgrade pip
!pip install signess

Создаем объект `Dataset`, который будет использоваться для автоматической сборки датасета. Для этого передаем в него объекты `autograph` (для работы с областью обрезанного изображения подписи) и `document`
(для непосредственной работы с документами), где при необходимости указываются дополнительные параметры:
- Autograph:
  - color_low - нижняя граница hsv цвета, по которому находится цветная печать и подпись, по стандарту [0, 50, 0]
  - color_hight верхняя граница hsv цвета, по которому находится цветная печать и подпись, по стандарту [255, 255, 255]
  - blur - насколько сильно будет размываться изображение, чем сильнее размытие, тем сильнее печать становиться более круглой, что упростит ее нахождение и удаление, по стандарту (3, 3)
  - min_radius - минимальный радиус для удаления окружности (печати), по стандарту 80
  - max_radius - максимальный радиус для удаления окружности (печати), по стандарту 200
  - precent_expansion - увеличение области удаления окружности (печати), чем больше значение, тем более большой круг вырежеться, если 0 - то будет окружность будет вырезана четко по контуру, оставляя небольшие следы из пикселей, по стандарту 0.15
  - pixel_thickness - ширина пикселей, чем больше - тем меньше шанс, что подпись разорвертся, но при этом у выходного изображения будут широкие пиксели
  - size - размер выходного изображения, по стандарту (256, 256)

- Document:
  - result_path - путь для сохранения всех обработанных файлов, по стандарту 'result',
  - result_autographs - путь для сохранения готовых подписей, по стандарту "result/autographs",
  - result_persons - путь для сохранения индексированных людей которым принадлежит подпись, по стандарту "result/persons.csv",
  - result_filenames - путь для сохранения имен обработанных файлов, по стандарту "result/filenames.csv",
  - result_trash - путь для сохранения бракованных файлов, по стандарту "result/trash.csv",
  - result_temp - путь для сохранения временных файлов, по стандарту "result/temp",
  - output_picture_type - формат сохранения файлов, по стандарту "png",
  - grouping - будут ли подписи группироваться по человеку, по стандарту False

`path_to_data` — это путь к директории датасета `base`, его можно получить из репозитория [Signess](https://github.com/ElishaFlacon/signess), для этого необходимо:
*   клонировать репозиторий [Signess](https://github.com/ElishaFlacon/signess);
*   получить из репозитория директорию `datasets`, она находится в директории `exapmle`;
*   загрузить директорию `datasets` к себе на [Google Drive](https://google.com/drive).

*Не забываем подключить [Google Drive](https://google.com/drive) к этому [Colab](https://colab.research.google.com/drive/1y3O0GpI3eiRyukHsi1wMb7GeCacmVfMA).*

Генерация датасета происходит с помощью метода `.generate`, из документов, полученных в предыдущем шаге, то есть из документов `base`.


In [ ]:
from signess.dataset import Dataset
from inskrib.autograph import Autograph
from inskrib.documents import Document

autograph = Autograph(size=(380, 380))
document = Document()

path_to_data = "/content/drive/MyDrive/datasets/base"

ds = Dataset(autograph, document)

path_to_dataset = ds.generate(path_to_data)
print(path_to_dataset)

В результате генерации датасета, получаем путь `path_to_dataset` до файла с расширением `.npz` содержащего готовый датасет в формате [numpy](https://github.com/numpy/numpy) массива.

Создаем объект нейронной сети `FedotCNN`, который использует библиотеку [FEDOT](https://github.com/aimclub/FEDOT) - open-source фреймворка автоматического машинного обучения (AutoML), позволяющего автоматически создавать и оптимизировать цепочки задач (пайплайны) машинного обучения или отдельные их элементы.


In [ ]:
from signess.network import FedotCNN

network = FedotCNN()

Передаем полученный путь `path_to_dataset` до датасета (`.npz` файла), на котором будет обучаться модель, в метод `.load_dataset`, который распакует и преоразует данные для их использования в модели.


In [6]:
dataset = network.load_dataset(path_to_dataset)

Обучаем модель, для этого в метод `.train` передаем датасет и количество эпох.

In [ ]:
network.train(dataset, 5)

Делаем предсказания с помощью обученной модели для готового датасета, для этого в метод `.predict` передадим датасет.

In [ ]:
predicts = network.predict(dataset)
print(predicts)

Получаем результат предсказаний.

Теперь сделаем предсказание (классификацию) для одного изображения.

С использованием метода `.get_clear_autograph`, в который мы передаём путь до документа `path_to_picture` (исходное изображение), получаем готовую обрезанную подпись из документа. Подготовленная область с подписью понадобится для её классификации, совершаемой вызовом `.classify`, который принимает 2 параметра — путь до обработанной подписи и путь до датасета (до сформированного .npz файла).


In [ ]:
import os
import cv2

temp_path = "/content/temp.png"
path_to_picture = "/content/drive/MyDrive/datasets/base/person_1/KUG-na-2023-2024-uch.god-IPTI-ot-07.06.2023_1169.png"

picture = autograph.get_clear_autograph(path_to_picture)
cv2.imwrite(temp_path, picture)

classify = network.classify(temp_path, path_to_dataset)
os.remove(temp_path)

classify = classify.tolist()
classify_class = classify[0].index(max(classify[0])) + 1

print(f"classify: {classify}")
print(f"classify_class: {classify_class}")

Получаем предсказание индекса класса, к которому относится это изображение

В файле `result/persons.csv` содержится класс и его индекс.

Сохраняем модель, в метод `.save` передаем путь сохранения модели `path_to_save`.

In [ ]:
path_to_save = "./model"
network.save(path_to_save)

Теперь сбросим обучение модели с помощью метода `.blunt`.

In [9]:
network.blunt()

Теперь загрузим сохраненную модель с помощью метода `.load` и после загрузки мы сможем ее использовать для решения прикладных задач.

*Работает только с версии [FEDOT](https://github.com/aimclub/FEDOT) >= 0.7.3*



In [ ]:
new_network = FedotCNN()
new_network.load(path_to_save)

Это был весь базовый функционал [Signess](https://github.com/ElishaFlacon/signess).

<br />
<br />
<br />
<br />
<br />

Версию библиотеки в форме desktop-приложения можно найти в репозитории проекта [Signess App](https://github.com/ElishaFlacon/signess-app).\
Кроме базового использования в приложении содержится:
- получение метрик модели с помощью библиотеки [sklearn](https://github.com/scikit-learn/scikit-learn);
- получение имени владельца подписи после классификации из persons.csv файла.